In [66]:
from pyspark.sql import SparkSession
from typing import Generator, List
import pandas as pd 
import os 

spark = SparkSession.builder.master("local").appName("year-taxi").getOrCreate()

directory: Generator[str, None, None] = (f"{os.getcwd()}/data/{i}" for i in range(1, len(os.listdir(f"{os.getcwd()}/data"))))
filename: List[str] = [f"{p}/{data}" for p in directory for data in os.listdir(p)]

In [68]:
import matplotlib.pyplot as plt 

qs: str = """
select 
    pickup_date ,count(*) as trips
from 
    (select split(pickup_datetime, " ")[0] as pickup_date from mobility_data)
group by
    pickup_date
order by
    pickup_date asc
"""

for lo in filename:
    d = spark.read.parquet(lo).createOrReplaceTempView("mobility_data")
    spark.sql(qs).write.csv("test.csv")


AnalysisException: path file:/Users/imhaneul/Documents/spark-kafka-distribute/sparkAnaliysis/test.csv already exists.